Extracting bbbc021 data. Run this script first just to load the data. I'd like to put all this into a simple plug-and-play, but the process of loading data is just too fiddly to easily put into a fuss-free UI.<br>

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import csv
import collections as coll
import re
#import os
import time
import math
import pandas as pd
import pickle
import random

#import toolz as tz
#from microscopium.screens import image_xpress
from skimage import io, img_as_float
from skimage.transform import downscale_local_mean
from sklearn.utils.extmath import cartesian as skcartesian

import skynet.bbbc021io as xio
import skynet.patch_extraction as pex
import skynet.utils as utils

data_path = "/Users/don/Documents/patch_data/"

In [2]:
#Load a db of plate-well, compound, concentration and moa
#Note that this database includes NaNs

labels_db = pd.read_csv("/Users/don/Documents/hcs/label_db2.csv",
                       usecols=[1,2,3,4])
#Don't load the first column; it's just a col of indices
#If the compound-concentration of some plate-well didn't have an moa,
#it won't be in this database

# Sample search
# result = xio.search_labels('BBBC021-40111-B03', labels_db)
# result

<font size = 5>Extracting Image data</font><br><br>
The main folder, 'BBBC021', has a bunch of subfolders, e.g. 'Week3_xxxx'.<br>
The platenumber is found in the subfolder name; that's the 'xxxx'.<br>
Each subfolder has a bunch of images; the well number is in the image name. <br><br>
Desired output:
 - A list of patches, each row comprises of 1 patch and its labels: [array:(20 x 20 x 3), plate-num, cc-label, moa-label]
 - Let's not unravel the patch yet.

In [3]:
#Declare some params:
n_patches = 100
patch_len = 120
ds_factor = (3,3,1)

# main_list is a list of all subfolder names
data_path = '/Users/don/Documents/patch_data/'
path = '/Users/don/Documents/BBBC021'
main_list = xio.get_main_list(path)

#csv_path = '/Users/don/Documents/BBBC021/BBBC021_parsed_metadata.csv'
#treatments = xio.get_labels_from_csv(csv_path, verbose=True)


arbitrary_img_path = path+"/Week1_22123/Week1_150607_B02_s1_w107447158-AC76-4844-8431-E6A954BD1174.jpeg"
test_img = io.imread(arbitrary_img_path)
im_h, im_w, ch = test_img.shape
print(test_img.shape)

print("No. of grids = ",pex.count_grids(patch_len, im_h, im_w))
#plt.imshow(test_img)

(2048, 2560, 3)
No. of grids =  357


In [4]:
#Takes ~20 to 30 mins
t0 = time.time()
main_data_list = []

idx = 1
img_counter = 0
for sf in main_list:
    t_i = time.time()
    subpath = path + '/' + sf
    print("%s of %s subfolders..." % (idx, len(main_list))
          , end="")
    sf_data_list_temp = xio.get_subfolder_patch_data(subpath, 
                                                     n_patches, 
                                                     patch_len, 
                                                     labels_db, 
                                                     ds=ds_factor,
                                                     verbose=1)
    print("%s images retrieved. " % len(sf_data_list_temp), end="")
    img_counter = img_counter + len(sf_data_list_temp)
    main_data_list = main_data_list + sf_data_list_temp
    idx +=1
    print("Runtime = %.2fs" % (time.time() - t_i))

data_list = np.array(main_data_list)
del main_data_list

dt = time.time() - t0
m, s = divmod(dt, 60)
h, m = divmod(m, 60)
print("")
print("All done in %d:%02d:%02d" % (h, m, s))
print("%s images used" % img_counter)
#Maybe remove all subfolder-level comments in get_subfolder_patch_data?

1 of 55 subfolders...60 images found in subfolder
20 images retrieved. Runtime = 29.48s
2 of 55 subfolders...60 images found in subfolder
20 images retrieved. Runtime = 31.63s
3 of 55 subfolders...60 images found in subfolder
20 images retrieved. Runtime = 30.40s
4 of 55 subfolders...60 images found in subfolder
25 images retrieved. Runtime = 29.88s
5 of 55 subfolders...60 images found in subfolder
25 images retrieved. Runtime = 28.09s
6 of 55 subfolders...60 images found in subfolder
25 images retrieved. Runtime = 29.00s
7 of 55 subfolders...60 images found in subfolder
18 images retrieved. Runtime = 31.09s
8 of 55 subfolders...60 images found in subfolder
18 images retrieved. Runtime = 28.35s
9 of 55 subfolders...60 images found in subfolder
18 images retrieved. Runtime = 30.01s
10 of 55 subfolders...60 images found in subfolder
16 images retrieved. Runtime = 30.70s
11 of 55 subfolders...60 images found in subfolder
16 images retrieved. Runtime = 30.10s
12 of 55 subfolders...60 image

<font size = 5><b>Loading Block</b></font>
<br>Load the p_list that you want.

In [ ]:
p_list = pickle.load(open(data_path+'d_ls0.p', 'rb'))
print(p_list.shape)
print(p_list[:,0][0].shape)
#p_list[:3]

<font size = 5><b>Optional step: Remove Overrepresented Classes</b></font><br> Remove compounds <i>taxol</i> and DMSO, or just DMSO.

In [5]:
d_list_filtered = []

for row in data_list:
    if row[2] != 'DMSO':
        d_list_filtered.append(row)

print(len(d_list_filtered))
print(d_list_filtered[0][0].shape)

# Add an image index
for i in range(len(d_list_filtered)):
    new_row = np.append(d_list_filtered[i], i)
    d_list_filtered[i] = new_row

d_list_filtered = np.array(d_list_filtered)
print(d_list_filtered.shape)

632
(100, 40, 40, 3)
(632, 6)


In [6]:
# Check no. of images
pw_names = []
img_idx_ls = []
for i in range(len(d_list_filtered)):
    pw_names.append(d_list_filtered[i][1])
    img_idx_ls.append(d_list_filtered[i][5])

print(len(d_list_filtered))
print(len(list(set(pw_names))))
print(len(list(set(img_idx_ls))))

632
632
632


<font size = 5><b>Equalize Class Proportions at Img Level</b></font><br>This is the first randomization step. I was hoping to be able to save all image data, but I don't think we can k-fold anything, because the smallest class has too few images.

In [7]:
# Get the moa labels, moa_ls
moa_ls = []
for img in d_list_filtered:
    moa_ls.append(img[4])
moa_ls = np.array(moa_ls)

# Get the different class names: moa_labels
moa_labels = list(set(moa_ls))
moa_labels.sort()
print("No. of classes = %s" % len(moa_labels))

# Get the indices of each class
# idx is a list of lists s.t. idx[i] = list of the indices of samples in class i
class_idx = []
for i in range(len(moa_labels)):
    class_idx.append(np.where(moa_ls==moa_labels[i])[0])

print("Class sizes:")
for i in range(len(class_idx)):
    print("%s: %s" % (i, len(class_idx[i])))

df = np.zeros(len(moa_labels))
for i in range(len(df)):
    df[i] = len(class_idx[i])
nb_small = int(min(df))
print("size of smallest class = %s" % nb_small)

# For each of the other classes, choose min_class observations at random
# sample is a list of the chosen indices
# Note that when it comes to the actual smallest class, choosing, say, 10 samples
# out of 10 possible choices will just return the whole class
sample = []
for i in range(len(class_idx)):
    subsample = np.random.choice(class_idx[i], nb_small, replace=False)
    sample.append(subsample)
sample = np.array(sample)
sample = sample.flatten()
print(len(sample))
print(len(list(set(sample))))

No. of classes = 12
Class sizes:
0: 15
1: 36
2: 18
3: 27
4: 24
5: 36
6: 22
7: 10
8: 42
9: 357
10: 21
11: 24
size of smallest class = 10
120
120


In [8]:
d_list2 = d_list_filtered[sample]
print(d_list2.shape)
print(d_list2[0][0].shape)

(120, 6)
(100, 40, 40, 3)


In [9]:
# Check no. of images, number of pw_names, no. of img_idx's
pw_names = []
img_idx_ls = []
for i in range(len(d_list2)):
    pw_names.append(d_list2[i][1])
    img_idx_ls.append(d_list2[i][5])

print(len(d_list2))
print(len(list(set(pw_names))))
print(len(list(set(img_idx_ls))))

120
120
120


<font size = 4><b>Data Formatting</b></font><br>
Change from image-level to patch-level format. Downscale if necessary.

In [10]:
p_list = []

for row in d_list2:
    patches = row[0]
    for patch in patches:
        record = [patch, row[1], row[2], row[3], row[4], row[5]]
        p_list.append(record)

p_list=np.array(p_list)
print(p_list.shape)
print(p_list[0].shape)
print(p_list[0][0].shape)

(12000, 6)
(6,)
(40, 40, 3)


In [11]:
colnames = ['img', 'pw', 'cpd', 'cc', 'moa', 'img_idx']
df = pd.DataFrame(p_list)
df.columns = colnames
print(df.shape)

(12000, 6)


In [12]:
pickle.dump(df, open(data_path+'p2.p', 'wb'))

<font size = 5><b>Save Images/Patches to a Folder for Viz</b></font><br>
Let's make a quilt too.

In [ ]:
from PIL import Image
from io import BytesIO
from skimage import img_as_ubyte

short_forms = {'Actin disruptors':'actin_d',
              'Aurora kinase inhibitors': 'ak_inhib',
              'Cholesterol-lowering': 'chol_l',
              'DNA damage': 'dna_dmg',
              'DNA replication': 'dna_repl',
              'Eg5 inhibitors': 'eg5',
              'Epithelial': 'epi',
              'Kinase inhibitors': 'k_inhib',
              'Microtubule destabilizers': 'mt_dest',
              'Microtubule stabilizers': 'mt_stab',
              'Protein degradation': 'prot_deg',
              'Protein synthesis': 'prot_syn',
              }

img_data = np.array(list(d0['img']))
img_idx_labels = list(d0['img_idx'])
#moa_labels = list(d0['moa'])
img_idx_ref = list(set(img_idx_labels))
print(img_data.shape)

save_fn = "/Users/don/Desktop/patches_g10s_r1/"
quilts = []
quilt_w = 10
n_patches = 80
p_len = 50
n_quilt_rows = int(n_patches/quilt_w)

for idx in img_idx_ref:
    d_temp = d0.loc[d0['img_idx'] == idx]
    img_data = np.array(list(d_temp['img']))
    moa = list(d_temp['moa'])[0]
    moa_s = short_forms[moa]
    pw = list(d_temp['pw'])[0]
    pw = pw[8:]
    cpd = list(d_temp['cpd'])[0]
    cc = list(d_temp['cc'])[0]
    if cpd == 'mevinolin/lovastatin':
        cpd = 'mevinolin.lovastatin'
    cpd_cc = str(cpd)+str(cc)
    
    grid = []
    for j in range(0, n_patches, quilt_w):
        row = img_data[j:j+quilt_w]
        row = np.vstack(row)
        grid.append(row)
    
    grid = np.hstack(np.array(grid))
    img_fn = moa_s+'_'+cpd_cc+'_'+pw+'_'+str(idx)+'.png'
    #print(img_fn)
    save_str = save_fn+img_fn
    io.imsave(save_str, grid)

In [ ]:
d_temp = d0.loc[d0['img_idx'] == 515]
print(d_temp.shape)
cpd = list(d_temp['pw'])[0]
cpd = cpd[8:]

<font size = 4><b>Equalize Class Proportions</b></font>

In [ ]:
# Get equal representation of classes at MOA/MOAx level
# First, let's see what the class sizes are like from a p_list

labels = []
for row in p_list:
    labels.append(row[3])
labels = np.array(labels)
labels_ref = list(set(labels))
labels_ref.sort()
nb_classes = len(labels_ref)
print("No. of classes = %s" % nb_classes)

#Get a DF of the labels
df = []
for i in range(len(labels_ref)):
    df.append(list(labels).count(labels_ref[i]))

df = np.array(df)
df_n = df/sum(df) * 100
min_class = min(df)
print("Size of smallest class = %s" % min_class)
print(df_n)

In [ ]:
idx = []
for i in range(len(labels_ref)):
    idx.append(np.where(labels==labels_ref[i])[0])

# For each of the other classes, choose min_class observations at random
sample = []
for i in range(len(idx)):
    subsample = np.random.choice(idx[i], min_class)
    sample.append(subsample)
sample = np.array(sample)
sample = sample.flatten()

# Extract the final p_list
p_list_eq = []
for i in range(len(sample)):
    p_list_eq.append(p_list[sample[i]])

print(len(p_list_eq))

In [ ]:
pickle.dump(p_list_eq, open(data_path+'p_list_g6s_eq.p', 'wb'))

In [ ]:
n_patches = p_list_x.shape[0]
batch = int(n_patches/5)

In [ ]:
pickle.dump(p_list_x[:batch], open(data_path+'p_list_f3_p1.p', 'wb'))
pickle.dump(p_list_x[batch:(2*batch)], open(data_path+'p_list_f3_p2.p', 'wb'))
pickle.dump(p_list_x[(2*batch):(3*batch)], open(data_path+'p_list_f3_p3.p', 'wb'))
pickle.dump(p_list_x[(3*batch):(4*batch)], open(data_path+'p_list_f3_p4.p', 'wb'))
pickle.dump(p_list_x[(4*batch):], open(data_path+'p_list_f3_p5.p', 'wb'))

In [ ]:
p_list = pickle.load(open(data_path+'p_list_g8.p', 'rb'))
print(p_list.shape)
print(p_list[0][0].shape)

In [ ]:
# Let's cut that up into 4 p_lists, to pickle.dump individually.
n_records = len(p_list)
idx1 = int(n_records / 4)
idx2 = idx1 * 2
idx3 = idx1 * 3

subset1 = p_list[:idx1]
subset2 = p_list[idx1:idx2]
subset3 = p_list[idx2:idx3]
subset4 = p_list[idx3:]

del p_list

In [ ]:
# Save them individually
pickle.dump(subset1, open('subset1.p', 'wb'))
print("subset1 pickled.dumped.")
pickle.dump(subset2, open('subset2.p', 'wb'))
print("subset2 pickled.dumped.")
pickle.dump(subset3, open('subset3.p', 'wb'))
print("subset3 pickled.dumped.")
pickle.dump(subset4, open('subset4.p', 'wb'))
print("subset4 pickled.dumped.")

In [ ]:
#np.save('p_list2', p_list)
pickle.dump(p_list, open('p_list0.p', 'wb'))

Image-level data:
- data_list.p: 10 (30, 30) patches per image

Patch-level data:
- p_list: 50 (30, 30, 3) patches, 1.25Gb
- p_list0: 10 (30, 30, 3) patches, 285Mb
- p_list2: 100 (30, 30, 3) patches, 2.08Gb

'f<x>' suffix indicates that DMSO and taxol compounds are removed.
- p_list_f0: 300 (30, 30, 3) patches 
- p_list_f1: 100 (30, 30, 3) patches
- p_list_f2: 50 (30, 30, 3) patches
- p_list_f3_p1 to ..._p5: 50 (100, 100, 3) patches

'g' suffix indicates that only DMSO has been removed. 
- p_list_g0: 10 (100, 100, 3) patches
- p_list_g1: 10 (300, 300, 3) patches
- p_list_g2: 100 (30, 30, 3) patches
- p_list_g4: 10 (400, 400, 3) patches
- p_list_g5_p1 to ..._p5: 50 (100, 100, 3) patches
- p_list_g0s_eq: length=12000, shape(50, 50, 3), i.e. 1000 patches per class. Mapping to the image level will not really be possible now, since equalizing class proportions was done at patch level. 

In [ ]:
p_list = pickle.load(open('p_list3.p', 'rb'))
print(p_list.shape)

Our data is currently in the format of an np array, shape(n_images, 5)
- mydata[0]: all n_patches patches extracted from each image. Shape (n_patches, patchlen, patchlen, 3)
- mydata[1]: plate-well coords of image
- mydata[2]: compound
- mydata[3]: concentration
- mydata[4]: moa

Now let's partition into training and testing data, according to a ratio specified above. Desired output: a list of length 4:
- list[0]: training patches
- list[1]: moa label of training patches
- list[2]: testing patches
- list[3]: moa label of testing patches